# Median Filter Drizzled Images

Due to new DQ arrays and reference files. The new ACS images have many NaN holes in the image. This notebook is designed to do a median filter over the image to fill in those holes.

In [ ]:
# Python Imports
from os import path
from glob import glob, iglob

# 3rd Party Imports
from tqdm.notebook import tqdm
import numpy as np
from scipy import ndimage
from skimage.restoration import inpaint_biharmonic as ibh

# Astropy Imports
from astropy.io import fits
from astropy.wcs import WCS
from regions import Regions

## Notebook Setup

In [ ]:
# Numpy Setup
_ = np.seterr(all='ignore')

In [ ]:
# Directories
IN_DIR  = path.abspath('../DrizzledImages')
OUT_DIR = path.abspath('../MedianFilteredImages')
DS9_DIR = path.abspath('../DS9/FOVs')

In [ ]:
# Filter Parameters
FILTER_SIZE = 5

# Filter Function
FILTER_FUNCTION = np.nanmedian

## Process Images

In [ ]:
# Loop through Images
for fileName in tqdm(glob(path.join(IN_DIR, '*_drc.fits.gz'))):
    
    # Get Output Name
    outName = path.basename(fileName)
    outName = outName.replace('.fits.gz', '_medfilt.fits.gz')
    outName = path.join(OUT_DIR, outName)
    
    # Open the File for Saving
    with fits.open(fileName) as hduList:
        
        # Get Instrument
        instr = hduList[0].header['PRIMESI'].strip()
        
        # Extract Image
        img = hduList[1].data
        
        # Load Regions
        regs = Regions.read(path.join(DS9_DIR, instr+'-FOV.reg'), format='ds9')
        fov  = regs[0].to_pixel(WCS(hduList[1]))
        fovMsk = fov.to_mask().to_image(img.shape).astype(bool)
        
        # Mask of Pixels to Replace
        # Replace Pixels in the FOV that are NaN
        repPixMsk = fovMsk & np.isnan(img)
        
        # Filter the Image
        filtImg = ndimage.generic_filter(
            img,
            function=FILTER_FUNCTION,
            size=FILTER_SIZE
        )
        img[repPixMsk] = filtImg[repPixMsk]
        
        # Store to New Image
        outList = hduList.copy()
        outList[0].header.add_history('NaN pixels in FOV replaced with Median Mask')
        outList['SCI'].data = img
        outList.writeto(outName, overwrite=True)

In [ ]:
# Loop through Images
for fileName in tqdm(glob(path.join(IN_DIR, '*_drc.fits.gz'))):
    
    # Get Output Name
    outName = path.basename(fileName)
    outName = outName.replace('.fits.gz', '_ibh.fits.gz')
    outName = path.join(OUT_DIR, outName)
    
    # Open the File for Saving
    with fits.open(fileName) as hduList:
        
        # Get Instrument
        instr = hduList[0].header['PRIMESI'].strip()
        
        # Extract Image
        img = hduList[1].data
        
        # Load Regions
        regs = Regions.read(path.join(DS9_DIR, instr+'-FOV.reg'), format='ds9')
        fov  = regs[0].to_pixel(WCS(hduList[1]))
        fovMsk = fov.to_mask().to_image(img.shape).astype(bool)
        
        # Mask of Pixels to Replace
        # Replace Pixels in the FOV that are NaN
        repPixMsk = fovMsk & np.isnan(img)
        
        # New Image with NaNs replaced with zeros
        zeroedImg = img.copy()
        zeroedImg[np.isnan(img)] = 0
        
        # Filter the Image
        filtImg = ibh(
            zeroedImg,
            mask=repPixMsk
        )
        
        # Re-NaN the Border
        filtImg[~fovMsk & np.isnan(img)] = np.NaN
        
        # Store to New Image
        outList = hduList.copy()
        outList[0].header.add_history('NaN pixels in FOV replaced with Median Mask')
        outList['SCI'].data = filtImg
        outList.writeto(outName, overwrite=True)